In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sklearn.metrics as metrics

import numpy as np
import random
import math

import matplotlib.pyplot as plt

import h5py

###FIX NUMPY LOAD FOR DICTIONARIES
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [17]:
with h5py.File("images_raw_doric_round1.h5", "r") as f:
    tmp_frames = f["/frames/raw"][:100]
    tmp_labels = f["/labels"][:100]
print(tmp_labels)

<HDF5 dataset "labels": shape (2111489,), type "<i2">


In [22]:
print(np.random.normal(0,1,(8,8,80,80)))

[[[[-1.07215588e+00  1.47073628e+00  6.85968033e-01 ... -9.97474509e-01
    -8.54439768e-01 -1.22554023e+00]
   [ 1.78225344e+00 -2.31657575e-01  8.43752277e-01 ...  2.02686624e+00
     1.13657360e+00  1.25577227e+00]
   [ 3.62488787e-01 -7.34431453e-01 -7.30943372e-01 ... -1.23184168e+00
    -1.64301704e-01  9.39730990e-01]
   ...
   [ 9.91510324e-02  6.44162283e-03 -8.94792339e-01 ...  5.15920808e-01
     2.12537147e+00  1.01018815e+00]
   [ 1.92468602e+00 -7.00723691e-02  2.66456182e+00 ...  7.68645226e-01
     9.16042179e-01  2.43486597e+00]
   [ 1.04429413e+00 -1.43119568e+00  2.13005332e-01 ...  1.75560590e+00
    -9.11915386e-01 -9.30361140e-02]]

  [[ 1.54490189e+00 -1.10918312e+00  5.86463465e-01 ...  1.28487900e+00
    -1.26029899e+00  6.66275298e-02]
   [-1.27282757e-01  8.44404020e-01 -8.33039945e-01 ...  2.28800462e-01
     1.70749878e-02  1.57874584e+00]
   [ 5.15810607e-01 -7.68371109e-01 -2.49423194e-01 ...  1.36851271e+00
     7.80375141e-01  5.02392499e-01]
   ...
   

In [33]:
a = np.arange(0,9)
print(a)
a = a[:len(a)-(len(a)%5)]
print(a)

[0 1 2 3 4 5 6 7 8]
[0 1 2 3 4]


In [31]:
np.reshape(a,(-1,2))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

In [40]:
with h5py.File("images_raw_doric_round1.h5",'r') as f:
    loaded = f["/frames/raw"][2000000:]
    test_data = np.zeros((len(loaded),8,80,80,1))
    cur_amt = 0
    index = 0
    for i in range(0, len(loaded)):
        test_data[index][cur_amt]=(loaded[i])[...,None]
        cur_amt+=1
        if cur_amt >= 8:
            index+=1
            cur_amt=0
    test_labels = f["/labels"][2000000:]
    test_labels = test_labels[:len(test_labels)-(len(test_labels)%8)]
    test_labels = test_labels[::8]

In [41]:
print(test_labels)

[44 44 23 ... 18 28  5]


In [45]:
a = [0,0,0,1,1,1]
b = [0,0,0,0,1,1]
metrics.confusion_matrix(a,b)

array([[3, 0],
       [1, 2]])

In [70]:
with h5py.File("images_raw_doric_round1.h5",'r') as f:
    freqs=np.zeros(48,dtype='int')
    length =len(f["/labels"])
    slice_len = length/100
    negs = 0
    for i in range(0,100):
        if (i+1)*slice_len > length:
            label = f["/labels"][int(i*slice_len):length]
        else:
            label = f["/labels"][int(i*slice_len):int((i+1)*slice_len)]
        for j in range(0,len(label)):
            if label[j]>=0:
                freqs[label[j]]+=1
            else:
                negs+=1
        #if i%10000==0:
        #    print((i*100/length),'          ',end='\r')
    print(freqs,negs)

[ 44641  48414  43926  54862  61511  52357  34839  93971  85881  45835
  58740  27863 118349  42785  81251  53469  30043  43119  28288  20868
  67811 113574  29424  35446  28466  56894  34009  40784  65822  41396
  45999  60144  16354  96451  13993  65968  32682  22802   8499  38738
  20413  18844  25242  36183  24067    100    159    157] 56


In [76]:
!nvidia-smi

Wed Jul 24 13:14:58 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.48                 Driver Version: 410.48                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:04:00.0 Off |                    0 |
| N/A   69C    P0   134W / 149W |  10943MiB / 11441MiB |     80%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00000000:05:00.0 Off |                    0 |
| N/A   55C    P0   139W / 149W |  10941MiB / 11441MiB |     85%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T